In [1]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [2]:
import sys
assert sys.version_info >= (3, 5)


import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "classification"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [4]:
import pandas as pd
import os
import tarfile
import urllib.request

ts=pd.read_csv("Features_testset.csv")
tv=pd.read_csv("Features_trainvalidation.csv")

In [5]:
tv.head()

,NComp,Comp_L2Norm,Comp_L3Norm,Comp_L5Norm,Comp_L7Norm,Comp_L10Norm,mean_Number,maxdiff_Number,dev_Number,max_Number,...,frac_sValence,frac_pValence,frac_dValence,frac_fValence,CanFormIonic,MaxIonicChar,MeanIonicChar,Index,TotalEnergy,Bandgap
0,1,1.000000,1.000000,1.000000,1.000000,1.000000,89.000000,0,0.000000,89,...,0.666667,0.0,0.333333,0.000000,0,0.000000,0.00000,1,-4.4803,0.0
1,1,1.000000,1.000000,1.000000,1.000000,1.000000,47.000000,0,0.000000,47,...,0.090909,0.0,0.909091,0.000000,0,0.000000,0.00000,5,-3.2681,0.0
2,2,0.745356,0.693361,0.670782,0.667408,0.666732,57.666667,32,14.222222,79,...,0.063830,0.0,0.638298,0.297872,0,0.088829,0.03948,9,-3.5294,0.0
3,2,0.745356,0.693361,0.670782,0.667408,0.666732,57.666667,32,14.222222,79,...,0.063830,0.0,0.638298,0.297872,0,0.088829,0.03948,13,-3.5643,0.0
4,2,0.745356,0.693361,0.670782,0.667408,0.666732,57.666667,32,14.222222,79,...,0.063830,0.0,0.638298,0.297872,0,0.088829,0.03948,17,-3.4825,0.0


In [6]:
tv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44174 entries, 0 to 44173
Columns: 148 entries, NComp to Bandgap
dtypes: float64(97), int64(51)
memory usage: 49.9 MB


In [7]:
tv.describe()

,NComp,Comp_L2Norm,Comp_L3Norm,Comp_L5Norm,Comp_L7Norm,Comp_L10Norm,mean_Number,maxdiff_Number,dev_Number,max_Number,...,frac_sValence,frac_pValence,frac_dValence,frac_fValence,CanFormIonic,MaxIonicChar,MeanIonicChar,Index,TotalEnergy,Bandgap
count,44174.000000,44174.000000,44174.000000,44174.000000,44174.000000,44174.000000,44174.000000,44174.000000,44174.000000,44174.000000,...,44174.000000,44174.000000,44174.000000,44174.000000,44174.000000,44174.000000,44174.000000,44174.000000,44174.000000,44174.000000
mean,2.921968,0.649572,0.584268,0.551721,0.543117,0.538253,39.136690,40.279486,14.351893,61.749264,...,0.225878,0.132151,0.439213,0.202758,0.215715,0.286594,0.108304,88347.000000,-5.720376,0.427692
std,0.572382,0.076830,0.097160,0.112877,0.119063,0.123463,16.441628,20.536967,7.818007,19.114566,...,0.145958,0.161999,0.228714,0.213150,0.411322,0.248945,0.102115,51008.318929,2.230297,1.196584
min,1.000000,0.447214,0.341995,0.275946,0.251700,0.234924,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-15.154200,0.000000
25%,3.000000,0.600000,0.517872,0.481796,0.467168,0.459210,26.587500,25.000000,8.375925,48.000000,...,0.131579,0.000000,0.280000,0.000000,0.000000,0.097332,0.033707,44174.000000,-7.222000,0.000000
50%,3.000000,0.612372,0.538609,0.506099,0.501109,0.500098,38.000000,39.000000,13.719504,65.000000,...,0.187500,0.064516,0.444444,0.157003,0.000000,0.213451,0.073640,88347.000000,-5.601150,0.000000
75%,3.000000,0.707107,0.642563,0.615030,0.604895,0.601032,50.500000,56.000000,19.555556,78.000000,...,0.279070,0.195652,0.612903,0.384615,0.000000,0.391662,0.149160,132520.000000,-4.045900,0.000000
max,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000,94.000000,93.000000,43.500000,94.000000,...,1.000000,0.714286,1.000000,0.866667,1.000000,0.921450,0.460725,176693.000000,-0.041300,10.117100


In [8]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
tv.head()

,NComp,Comp_L2Norm,Comp_L3Norm,Comp_L5Norm,Comp_L7Norm,Comp_L10Norm,mean_Number,maxdiff_Number,dev_Number,max_Number,...,frac_sValence,frac_pValence,frac_dValence,frac_fValence,CanFormIonic,MaxIonicChar,MeanIonicChar,Index,TotalEnergy,Bandgap
0,1,1.000000,1.000000,1.000000,1.000000,1.000000,89.000000,0,0.000000,89,...,0.666667,0.0,0.333333,0.000000,0,0.000000,0.00000,1,-4.4803,0.0
1,1,1.000000,1.000000,1.000000,1.000000,1.000000,47.000000,0,0.000000,47,...,0.090909,0.0,0.909091,0.000000,0,0.000000,0.00000,5,-3.2681,0.0
2,2,0.745356,0.693361,0.670782,0.667408,0.666732,57.666667,32,14.222222,79,...,0.063830,0.0,0.638298,0.297872,0,0.088829,0.03948,9,-3.5294,0.0
3,2,0.745356,0.693361,0.670782,0.667408,0.666732,57.666667,32,14.222222,79,...,0.063830,0.0,0.638298,0.297872,0,0.088829,0.03948,13,-3.5643,0.0
4,2,0.745356,0.693361,0.670782,0.667408,0.666732,57.666667,32,14.222222,79,...,0.063830,0.0,0.638298,0.297872,0,0.088829,0.03948,17,-3.4825,0.0


In [10]:
df=pd.DataFrame(tv)
df.shape

(44174, 148)

# Keras

In [11]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(tv, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

28271 train examples
7068 validation examples
8835 test examples


In [12]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Bandgap')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [13]:
type(tv)

pandas.core.frame.DataFrame

In [14]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

In [15]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))

Every feature: ['NComp', 'Comp_L2Norm', 'Comp_L3Norm', 'Comp_L5Norm', 'Comp_L7Norm', 'Comp_L10Norm', 'mean_Number', 'maxdiff_Number', 'dev_Number', 'max_Number', 'min_Number', 'most_Number', 'mean_MendeleevNumber', 'maxdiff_MendeleevNumber', 'dev_MendeleevNumber', 'max_MendeleevNumber', 'min_MendeleevNumber', 'most_MendeleevNumber', 'mean_AtomicWeight', 'maxdiff_AtomicWeight', 'dev_AtomicWeight', 'max_AtomicWeight', 'min_AtomicWeight', 'most_AtomicWeight', 'mean_MeltingT', 'maxdiff_MeltingT', 'dev_MeltingT', 'max_MeltingT', 'min_MeltingT', 'most_MeltingT', 'mean_Column', 'maxdiff_Column', 'dev_Column', 'max_Column', 'min_Column', 'most_Column', 'mean_Row', 'maxdiff_Row', 'dev_Row', 'max_Row', 'min_Row', 'most_Row', 'mean_CovalentRadius', 'maxdiff_CovalentRadius', 'dev_CovalentRadius', 'max_CovalentRadius', 'min_CovalentRadius', 'most_CovalentRadius', 'mean_Electronegativity', 'maxdiff_Electronegativity', 'dev_Electronegativity', 'max_Electronegativity', 'min_Electronegativity', 'most_E

# 전처리

In [19]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for our feature.
  normalizer = preprocessing.Normalization(axis=None)

  # Prepare a Dataset that only yields our feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [20]:
photo_count_col = train_features['NComp']
layer = get_normalization_layer('NComp', train_ds)
layer(photo_count_col)

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([ 0.12678103, -1.6173462 ,  1.8709081 ,  1.8709081 , -1.6173462 ],
      dtype=float32)>

In [21]:
#사용할 열 선택

batch_size = 256
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [22]:
all_inputs = []
encoded_features = []

# Numeric features.
for header in ['NComp', 'Comp_L2Norm', 'Comp_L3Norm', 'Comp_L5Norm', 'Comp_L7Norm', 'Comp_L10Norm', 'mean_Number', 'maxdiff_Number', 'dev_Number', 'max_Number', 'min_Number', 'most_Number', 'mean_MendeleevNumber', 'maxdiff_MendeleevNumber', 'dev_MendeleevNumber', 'max_MendeleevNumber', 'min_MendeleevNumber', 'most_MendeleevNumber', 'mean_AtomicWeight', 'maxdiff_AtomicWeight', 'dev_AtomicWeight', 'max_AtomicWeight', 'min_AtomicWeight', 'most_AtomicWeight', 'mean_MeltingT', 'maxdiff_MeltingT', 'dev_MeltingT', 'max_MeltingT', 'min_MeltingT', 'most_MeltingT', 'mean_Column', 'maxdiff_Column', 'dev_Column', 'max_Column', 'min_Column', 'most_Column', 'mean_Row', 'maxdiff_Row', 'dev_Row', 'max_Row', 'min_Row', 'most_Row', 'mean_CovalentRadius', 'maxdiff_CovalentRadius', 'dev_CovalentRadius', 'max_CovalentRadius', 'min_CovalentRadius', 'most_CovalentRadius', 'mean_Electronegativity', 'maxdiff_Electronegativity', 'dev_Electronegativity', 'max_Electronegativity', 'min_Electronegativity', 'most_Electronegativity', 'mean_NsValence', 'maxdiff_NsValence', 'dev_NsValence', 'max_NsValence', 'min_NsValence', 'most_NsValence', 'mean_NpValence', 'maxdiff_NpValence', 'dev_NpValence', 'max_NpValence', 'min_NpValence', 'most_NpValence', 'mean_NdValence', 'maxdiff_NdValence', 'dev_NdValence', 'max_NdValence', 'min_NdValence', 'most_NdValence', 'mean_NfValence', 'maxdiff_NfValence', 'dev_NfValence', 'max_NfValence', 'min_NfValence', 'most_NfValence', 'mean_NValance', 'maxdiff_NValance', 'dev_NValance', 'max_NValance', 'min_NValance', 'most_NValance', 'mean_NsUnfilled', 'maxdiff_NsUnfilled', 'dev_NsUnfilled', 'max_NsUnfilled', 'min_NsUnfilled', 'most_NsUnfilled', 'mean_NpUnfilled', 'maxdiff_NpUnfilled', 'dev_NpUnfilled', 'max_NpUnfilled', 'min_NpUnfilled', 'most_NpUnfilled', 'mean_NdUnfilled', 'maxdiff_NdUnfilled', 'dev_NdUnfilled', 'max_NdUnfilled', 'min_NdUnfilled', 'most_NdUnfilled', 'mean_NfUnfilled', 'maxdiff_NfUnfilled', 'dev_NfUnfilled', 'max_NfUnfilled', 'min_NfUnfilled', 'most_NfUnfilled', 'mean_NUnfilled', 'maxdiff_NUnfilled', 'dev_NUnfilled', 'max_NUnfilled', 'min_NUnfilled', 'most_NUnfilled', 'mean_GSvolume_pa', 'maxdiff_GSvolume_pa', 'dev_GSvolume_pa', 'max_GSvolume_pa', 'min_GSvolume_pa', 'most_GSvolume_pa', 'mean_GSbandgap', 'maxdiff_GSbandgap', 'dev_GSbandgap', 'max_GSbandgap', 'min_GSbandgap', 'most_GSbandgap', 'mean_GSmagmom', 'maxdiff_GSmagmom', 'dev_GSmagmom', 'max_GSmagmom', 'min_GSmagmom', 'most_GSmagmom', 'mean_SpaceGroupNumber', 'maxdiff_SpaceGroupNumber', 'dev_SpaceGroupNumber', 'max_SpaceGroupNumber', 'min_SpaceGroupNumber', 'most_SpaceGroupNumber', 'frac_sValence', 'frac_pValence', 'frac_dValence', 'frac_fValence', 'CanFormIonic', 'MaxIonicChar', 'MeanIonicChar']:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

In [23]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(all_inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [24]:
# rankdir='LR' is used to make the graph horizontal.
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [25]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10


C:\Users\tmd97\anaconda3\lib\site-packages\keras\engine\functional.py:559: UserWarning: Input dict contained keys ['Index', 'TotalEnergy'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


111/111 [==============================] - 8s 33ms/step - loss: -2.9980 - accuracy: 0.6486 - val_loss: -7.2110 - val_accuracy: 0.6624
Epoch 2/10
111/111 [==============================] - 4s 22ms/step - loss: -15.8732 - accuracy: 0.6521 - val_loss: -23.9307 - val_accuracy: 0.6611
Epoch 3/10
111/111 [==============================] - 4s 23ms/step - loss: -39.9568 - accuracy: 0.6535 - val_loss: -50.7544 - val_accuracy: 0.6614
Epoch 4/10
111/111 [==============================] - 4s 21ms/step - loss: -73.6400 - accuracy: 0.6561 - val_loss: -87.4114 - val_accuracy: 0.6633
Epoch 5/10
111/111 [==============================] - 4s 21ms/step - loss: -121.2566 - accuracy: 0.6565 - val_loss: -134.6990 - val_accuracy: 0.6636
Epoch 6/10
111/111 [==============================] - 4s 22ms/step - loss: -175.8963 - accuracy: 0.6503 - val_loss: -190.7254 - val_accuracy: 0.6611
Epoch 7/10
111/111 [==============================] - 4s 23ms/step - loss: -241.0344 - accuracy: 0.6146 - val_loss: -255.9041 -

In [26]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

35/35 [==============================] - 1s 20ms/step - loss: -464.7420 - accuracy: 0.6414
Accuracy 0.641426146030426
